In [1]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

from scipy.sparse import hstack

In [2]:
train_df = pd.read_csv("train_data.csv", index_col="Id_Записи", sep=";")

In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 61976 entries, 0 to 61975
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Id_Пациента       61976 non-null  int64 
 1   Возраст           61976 non-null  int64 
 2   Диагноз           61976 non-null  object
 3   Жалобы            61976 non-null  object
 4   Источник_рекламы  61976 non-null  object
 5   Клиника           61976 non-null  int64 
 6   Код_диагноза      61976 non-null  object
 7   Пол               61976 non-null  int64 
 8   Услуга            61976 non-null  object
dtypes: int64(4), object(5)
memory usage: 4.7+ MB


Nan значений нет, можно сразу приступать к анализу данных и построению модели.

In [4]:
train_df.head(10)

,Id_Пациента,Возраст,Диагноз,Жалобы,Источник_рекламы,Клиника,Код_диагноза,Пол,Услуга
Id_Записи,,,,,,,,,
0,115819,54,Гипертензивная болезнь сердца [гипертоническая...,"на повышение ад утром до 140/90 мм.рт.ст., пер...",Другое,5,I11,2,"Прием врача-кардиолога повторный, амбулаторный"
1,399973,32,Доброкачественное новообразование молочной железы,На наличие опухоли в левой молочной железе,Другое,3,D24,2,"Прием врача-онколога (маммолога), повторный, а..."
2,427563,72,Простой хронический бронхит,Активных жалоб нет.,Интернет,6,J41.0,2,Прием первичный врача-пульмонолога
3,257197,55,Другая дорсалгия,"на сохраняющиеся боли в спине и пояснице, сков...",Другое,3,M54.8,1,"Прием врача-невролога повторный, амбулаторный"
4,281066,28,Острый фарингит,"на дискомфорт в горле, слабое першение, слабость",Другое,3,J02,2,"Прием врача-оториноларинголога повторный, амбу..."
5,341445,46,Ушиб пальца(ев) кисти с повреждением ногтевой ...,на боли в 3 пальце правой кисти,Другое,3,S60.1,2,"Прием врача-хирурга первичный, амбулаторный"
6,416352,29,Поражение межпозвоночных дисков других отделов,Не изменились с момента первого приема,Интернет,2,M51,2,"Прием врача-невролога повторный, амбулаторный"
7,251280,38,Диффузная кистозная мастопатия,На боли в молочных железах по циклу,Другое,3,N60.1,2,"Прием врача-онколога (маммолога), повторный, а..."
8,208376,32,Другой хронический цистит,на боли в правой пахово-подвздошной области.,Интернет,5,N30.2,2,"Прием врача-уролога повторный, амбулаторный"


Отдельно посмотрим на диагнозы, которые нужно предсказать:

In [5]:
train_df["Диагноз"].value_counts().head(15)

Диагноз
Острая инфекция верхних дыхательных путей неуточненная                                                             2147
Остеохондроз позвоночника у взрослых                                                                               1949
Острый вагинит                                                                                                     1942
Острый назофарингит (насморк)                                                                                      1379
Беременность подтвержденная                                                                                        1358
Хронический простатит                                                                                              1111
Общий медицинский осмотр                                                                                           1048
Вагинит, вульвит и вульвовагинит при инфекционных и паразитарных болезнях, классифицированных в других рубриках     956
Гипертензивная болезнь сердца [г

Всего уникальных диагнозов 2286. Самый популярный диагноз встречается 2147 раз. А самый редкий - всего 1 раз.

Имеет смысл рассматривать точность предсказания не только для всех диагнозов, но и для 10 самых популярных диагнозов.


In [6]:
n_top = 10

top_diagnoses = train_df["Код_диагноза"].value_counts().nlargest(n_top).index
top_diagnoses

Index(['J06.9', 'M42.1', 'N76.0', 'J00', 'Z32.1', 'N41.1', 'Z00.0', 'I11',
       'N77.1*', 'K30'],
      dtype='object', name='Код_диагноза')

## Подготовка данных
Рассмотрим, на мой взляд, наиболее важные признаки для предсказания диагноза: "Жалобы", "Услуга" и "Возраст"

In [7]:
valid_size = 0.2
random_state = 42 # Для воспроизводимости результатов

Разобьем данные на обучающую и валидационную выборки. Проведем векторизацию текстовых признаков и нормализацию числовых:

In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(
    train_df[["Жалобы", "Услуга", "Возраст"]], 
    train_df["Код_диагноза"],
    test_size=valid_size,
    random_state=random_state
)

In [9]:
complaint_vectorizer = TfidfVectorizer()
X_train_complaints = complaint_vectorizer.fit_transform(X_train["Жалобы"])
X_valid_complaints = complaint_vectorizer.transform(X_valid["Жалобы"])

service_vectorizer = TfidfVectorizer()
X_train_service = service_vectorizer.fit_transform(X_train["Услуга"])
X_valid_service = service_vectorizer.transform(X_valid["Услуга"])

scaler = MinMaxScaler()
X_train_age = scaler.fit_transform(X_train[["Возраст"]])
X_valid_age = scaler.transform(X_valid[["Возраст"]])

In [10]:
X_train_transformed = hstack([X_train_complaints, X_train_service, X_train_age])
X_valid_transformed = hstack([X_valid_complaints, X_valid_service, X_valid_age])

In [11]:
print(f"X_train_transformed shape: {X_train_transformed.shape}")
print(f"X_valid_transformed shape: {X_valid_transformed.shape}")

del X_train, X_valid, X_train_complaints, X_valid_complaints, X_train_service, X_valid_service, X_train_age, X_valid_age

X_train_transformed shape: (49580, 12248)
X_valid_transformed shape: (12396, 12248)


## Обучение модели

In [12]:
clf = LogisticRegression(random_state=random_state, tol=0.004)
clf.fit(X_train_transformed, y_train)

LogisticRegression(random_state=42, tol=0.004)

## Метрика с учётом предсказания всех диагнозов

In [13]:
y_train_pred = clf.predict(X_train_transformed)
print(f"Train Accuracy: {round(accuracy_score(y_train, y_train_pred) * 100, 2)}%")

Train Accuracy: 34.65%


In [14]:
y_valid_pred = clf.predict(X_valid_transformed)
print(f"Valid Accuracy: {round(accuracy_score(y_valid, y_valid_pred) * 100, 2)}%")

Valid Accuracy: 32.49%


## Метрика с учётом предсказания 10 самых распространённых диагнозов

In [15]:
top_indices = y_valid.isin(top_diagnoses)
y_test_top = y_valid[top_indices]
y_pred_top = y_valid_pred[top_indices]

print(f"Valid Accuracy (Top {n_top}): {round(accuracy_score(y_test_top, y_pred_top) * 100, 2)}%")

Valid Accuracy (Top 10): 71.54%


Как видно, модель показывает хороший результат на тестовой выборке при предсказании распространённых диагнозов.

## TODO
Чтобы улучшить результат предсказания модели всех дианозов, включая не распространённые, можно попробовать обучать модель с учётом весов классов, чтобы учесть дисбаланс классов.

Также можно попробовать другие модели, регрессию или KNN.

Также можно обработать и использовать другие фичи из датасета - пол, источник рекламы, номер клиники и id пациента.

# Predict on test data
Подготовим тестовые данные и сделаем предсказание

In [16]:
test_df = pd.read_csv("test_data.csv", index_col="Id_Записи", sep=";")

In [17]:
X_test_complaints = complaint_vectorizer.transform(test_df["Жалобы"])
X_test_service = service_vectorizer.transform(test_df["Услуга"])
X_test_age = scaler.transform(test_df[["Возраст"]])

In [18]:
X_test_transformed = hstack([X_test_complaints, X_test_service, X_test_age])

## Предсказание и сохранение результата

In [19]:
test_df["Код_диагноза"] = clf.predict(X_test_transformed)

In [20]:
# добавим столбец с диагнозом, исходя из кода диагноза

# code_to_diagnosis: словарь код_диагноза -> диагноз
code_to_diagnosis = train_df.set_index("Код_диагноза")["Диагноз"].to_dict()

test_df["Диагноз"] = test_df["Код_диагноза"].map(code_to_diagnosis)

In [21]:
test_df.head(5)

,Id_Пациента,Возраст,Жалобы,Источник_рекламы,Клиника,Пол,Услуга,Код_диагноза,Диагноз
Id_Записи,,,,,,,,,
0,269166,61,"на затруднение носового дыхания, скудное слизи...",Другое,3,1,"Прием врача-оториноларинголога повторный, амбу...",J00,Острый назофарингит (насморк)
1,509296,27,"прежние, динамики в состоянии не отмечает",ДМС,3,2,"Прием врача-эндокринолога повторный, амбулаторный",Z01.8,Другое уточненное специальное обследование
2,418164,33,задержка мс. Тест на беременность положит (+),Интернет,3,2,"Прием врача-акушера-гинеколога первичный, амб...",Z01.4,Гинекологическое обследование (общее) (рутинное)
3,413186,30,"на бели, вздутие живота",Другое,3,2,"Прием врача-акушера-гинеколога повторный, амбу...",N76.0,Острый вагинит
4,257969,32,"на периодическое затруднение носового дыхания,...",Другое,3,2,"Прием врача-оториноларинголога повторный, амбу...",J01.0,Острый верхнечелюстной синусит


In [22]:
test_df.to_csv("result.csv", index="Id_Записи", sep=";")